<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/ExpandRetreat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import math
import pandas as pd
import plotly.graph_objects as go

In [16]:
def expandOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = 100.0 * (inf + top)/ bottom
  if ret> 100.0:
     ret = 100.0
  if ret<1.0:
     ret = 1.0
  return ret

def dailyIncrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [28]:
def retreatOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = inf - ((100 * ( (100-inf) + top)/ bottom) - (100-inf))
  if ret> 100.0:
     ret = 100.0
  if ret<1.0:
     ret = 1.0
  return ret

def dailyDecrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [24]:
retreatOneDay(9.2,2200)

7.3660866155447415

In [19]:
expandOneDay(65.9,2200)

72.6974398271657

In [20]:
def daysToPendingExpansion(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf < 75.0:
    day += 1
    tinf = expandOneDay(tinf, pop)
  return day

def daysToPendingRetreat(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf > 2.5:
    day += 1
    tinf = retreatOneDay(tinf, pop)
  return day

def daysToExpansion(inf: float, pop: int):
  ret = daysToPendingExpansion(inf,pop)
  ret += 1 # pending day
  ret += 6 # active expansion
  return ret

def daysToRetreat(inf: float, pop: int):
  ret = daysToPendingRetreat(inf,pop)
  ret += 1 # pending day
  ret += 6 # active retreat
  ret += 1
  return ret

In [21]:
daysToPendingExpansion(65.9,2200)

2

In [22]:
daysToPendingRetreat(65.9,2200)

15

In [54]:

def getRetreatFrame(inf: float, pop: int):
  dayNum = []
  infs = []
  day = 0

  dayNum.append(day)
  infs.append(inf)

  tinf = inf
  while tinf > 2.5:
    day +=1
    tinf = retreatOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)

  n = 0
  while n<=8:
    day +=1
    tinf = retreatOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)
    n += 1

  df = pd.DataFrame( list(zip(dayNum, infs)), columns=["dayNum","inf"])
  return df

def getAdvanceFrame(inf: float, pop: int, nDays: int):
  dayNum = []
  infs = []
  day = 0

  dayNum.append(day)
  infs.append(inf)

  tinf = inf
  while day<= nDays:
    day +=1
    tinf = expandOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)

  df = pd.DataFrame( list(zip(dayNum, infs)), columns=["dayNum","inf"])
  return df

In [66]:
inf = 36.0
pop = 36000

nDays = daysToRetreat(inf, pop)
df = getRetreatFrame(inf,pop)
df2 = getAdvanceFrame(inf, pop, nDays)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df2.dayNum, y=df2.inf,
                    mode='lines',
                    name='Ally',
                    marker_color="#0000ff"))
fig.add_trace(go.Scatter(x=df.dayNum, y=df.inf,
                    mode='lines',
                    name='Target',
                    marker_color="#ff0000"))
retLine = list( 2.5 for i in range(df.dayNum.size))
expLine = list( 75.0 for i in range(df.dayNum.size))
fig.add_trace(go.Scatter(x=df2.dayNum, y=retLine,
                    mode='lines',
                    name='Retreat',
                    marker_color="#ff0000"))
fig.add_trace(go.Scatter(x=df2.dayNum, y=expLine,
                    mode='lines',
                    name='Expand',
                    marker_color="#00ff00"))
fig.show()



In [59]:
df.dayNum.size

25